In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix

from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

## Non-Negative Matrix Factorization for Movie Recommendations

This appends to a previous week's assignment on recommender systems.

We have four files:
- rec_movies: one row per movie, its release year, and a number of columns for genre
- rec_users: one row per user, their gender, their age, their occupation, and their zip code
- rec_train: one row per user, movie, and rating
- rec_test: same as rec_train

The general idea here is that we start with the utility matrix with one row per user and one column per movies. The cells are ratings for that user and that movie. 

When we use non-negative matrix factorization (NMF), we decompose the utility matrix into the W and H matrices. The W matrix has one row per user. The H matrix has one row per movie. They share the same columns with each column representing a latent factor. A latent factor is an aspect of the data in the utility matrix encoded similar to how the word embeddings from Word2Vec created new meaningful features that we are not able to map one-to-one to something meaningful to us. NMF learns patterns in the utility matrix and encodes them in the features for W and H.

We then multiply W and H together with these patterns encoded in the latent factors. The result of that multiplication is a new utility matrix with the same dimensions, row meanings, and column meanings as the original. The catch is that multiplying W and H tells us what each cell is expected to be based on extending the patterns encoded in the latent factors. This lets us replace 0s with what we would expect to see based on those latent factors, allowing us to fill in the 0s with predictions.

From there, we can select a user and a movie we are interested in and look up the prediction in the new utility matrix. We loop through the test set and pull out predictions for each user and movie combination.

Singular value decomposition (SVD) works differently in terms of how it decomposes the utility matrix. I explained SVD in the BBC classification notebook. For this notebook, I am using SVD to see how similar or different its RMSE ends up being, mostly for curiosity and reference for NMF.

## 1. Load Moving Ratings Data and Predict with Matrix Factorization


In [2]:
users = pd.read_csv('data/rec_users.csv')
movies = pd.read_csv('data/rec_movies.csv')
train = pd.read_csv('data/rec_train.csv')
test = pd.read_csv('data/rec_test.csv')

The trickiest code here is the creation of the utility matrix. We read in the training dataset with one row per user, movie, and rating. We need to convert that to a utility matrix with one row per user and one column per movie. Each cell will be that user's rating for that movie. We build the utility matrix using `scipy` sparse matrix functionality with `coo_matrix()`.

In [3]:
allusers = list(users['uID'])
allmovies = list(movies['mID'])
genres = list(movies.columns.drop(['mID', 'title', 'year']))
mid2idx = dict(zip(movies.mID,list(range(len(movies)))))
uid2idx = dict(zip(users.uID,list(range(len(users)))))

# Turns the train set into a utility matrix with one row per user, one 
# column per movie, and each cell as that user's rating for that movie
movie_ratings_utility_matrix = np.array(
        coo_matrix(
            (
                list(train.rating)
                , (
                    [uid2idx[x] for x in train.uID]
                    , [mid2idx[x] for x in train.mID]
                )
            ), shape=(
                len(allusers)
                , len(allmovies)
            )
        ).toarray()
    )

As a reminder, root mean squared error (RMSE) takes the difference between each prediction and the actual value, squares that distance, find the mean of those squared distances, and then takes the square root of that mean. This gives us a measure of how far apart on average each prediction is from the corresponding observed value. Squaring the mean puts the RMSE measure back into the scale of the units of ratings, with the ratings being on a 0-5 integer scale.

In [4]:
def rmse(y_preds):
    y_preds[np.isnan(y_preds)] = 3 #In case there is nan values in prediction, it will impute to 3.
    y_true = np.array(test.rating)
    return np.sqrt(((y_true - y_preds)**2).mean())

Not including solutions here, but tested and confirmed that 
- `predict_everything_to_3()`
- `predict_to_user_average()`

works. Just to make sure my refactoring here did not mess anything up.

As a reminder, here is how each of the methods performed in the week three assignment.

| Method                              | RMSE  |
|:------------------------------------|:-----:|
| Baseline, $Y_p$=3                   | 1.259 |
| Baseline, $Y_p=\mu_u$               | 1.035 |
| Content based, item-item            | 1.013 |
| Collaborative, cosine               | 1.026 |
| Collaborative, jaccard, $M_r\geq 3$ | 0.982 |
| Collaborative, jaccard, $M_r\geq 1$ | 0.991 |
| Collaborative, jaccard, $M_r$       | 0.952 |

We will test out both NMF and SVD. The assignment asks to test out NMF, but I was curious how NMF compares to another matrix factorization algorithm.

We will need to toggle `max_iter` on for NMF after 15 components since the convergence for the estimated W and H matrices needs more than the default 200 iterations.

We test out both NMF and SVD with different numbers of features in the decomposition spanning from 5 to 100. RMSE summaries for each are below.

In [5]:
nmf = NMF(
    n_components=5
    # , max_iter=500
    , random_state=42
)
W = nmf.fit_transform(movie_ratings_utility_matrix)
H = nmf.components_

preds_nmf = W.dot(H)

In [6]:
print(f'W matrix shape: {W.shape}')
print(f'H matrix shape: {H.shape}')
print(f'Utility matrix shape: {movie_ratings_utility_matrix.shape}')

W matrix shape: (6040, 25)
H matrix shape: (25, 3883)
Utility matrix shape: (6040, 3883)


In [6]:
svd = TruncatedSVD(
    n_components=5
    , random_state=42
)

U = svd.fit_transform(movie_ratings_utility_matrix)
S = np.diag(svd.singular_values_)
V = svd.components_

preds_svd = U.dot(S.dot(V))

In [8]:
np.min(preds_nmf)

0.0

In [8]:
print(f'U matrix shape: {U.shape}')
print(f'S matrix shape: {S.shape}')
print(f'V matrix shape: {V.shape}')
print(f'Utility matrix shape: {movie_ratings_utility_matrix.shape}')

U matrix shape: (6040, 25)
S matrix shape: (25, 25)
V matrix shape: (25, 3883)
Utility matrix shape: (6040, 3883)


NMF and SVD are returning predictions outside the original 0-5 rating scale. They are also producing floats. We will rescale both back to 0-5 and then round the predictions to get integer ratings.

In [9]:
mms_nmf = MinMaxScaler((0, 5))
preds_test_nmf = np.array([
    preds_nmf[uid2idx[uid], mid2idx[mid]]
    for uid, mid in np.array(test.drop('rating', axis=1))
])
preds_test_nmf_scaled = mms_nmf.fit_transform(preds_test_nmf.reshape(-1,1)).flatten()
preds_test_nmf_scaled_rounded = np.round(preds_test_nmf_scaled)

mms_svd = MinMaxScaler((0, 5))
preds_test_svd = np.array([
    preds_svd[uid2idx[uid], mid2idx[mid]]
    for uid, mid in np.array(test.drop('rating', axis=1))
])
preds_test_svd_scaled = mms_svd.fit_transform(preds_test_svd.reshape(-1,1)).flatten()
preds_test_svd_scaled_rounded = np.round(preds_test_svd_scaled)

In [10]:
print(f'RMSE for NMF: {rmse(preds_test_nmf)}')
print(f'RMSE for SVD: {rmse(preds_test_svd)}')
print(f'RMSE for NMF scaled: {rmse(preds_test_nmf_scaled)}')
print(f'RMSE for SVD scaled: {rmse(preds_test_svd_scaled)}')
print(f'RMSE for NMF scaled and rounded: {rmse(preds_test_nmf_scaled_rounded)}')
print(f'RMSE for SVD scaled and rounded: {rmse(preds_test_svd_scaled_rounded)}')

RMSE for NMF: 2.8560903217001954
RMSE for SVD: 1316.2845908671795
RMSE for NMF scaled: 3.0760606462814613
RMSE for SVD scaled: 3.1834367157206906
RMSE for NMF scaled and rounded: 3.1341820543163386
RMSE for SVD scaled and rounded: 3.2588719543608233


## RMSE Summary

|       Algorithm        | Components: 5 | Components: 10 | Components: 15 | Components: 20 | Components: 25 & max_iter: 500 | Components: 50 & max_iter: 500 | Components: 100 & max_iter: 500 |
|:----------------------:|:--------------|:---------------|:---------------|:---------------|:-------------------------------|:-------------------------------|:--------------------------------|
|          NMF           | 2.991         | 2.912          | 2.873          | 2.862          | 2.856                          | 2.899                          | 3.008                           |
|          SVD           | 1296.983      | 1315.453       | 1318.607       | 1318.425       | 1316.285                       | 1295.586                       | 1266.619                        |
|       NMF Scaled       | 3.279         | 3.195          | 3.118          | 3.076          | 3.076                          | 3.076                          | 3.213                           |
|       SVD Scaled       | 3.263         | 3.213          | 3.199          | 3.19           | 3.183                          | 3.167                          | 3.171                           |
| NMF Scaled and Rounded | 3.362         | 3.266          | 3.181          | 3.134          | 3.134                          | 3.132                          | 3.273                           |
| SVD Scaled and Rounded | 3.351         | 3.294          | 3.278          | 3.266          | 3.259                          | 3.241                          | 3.243                           |



## 2. Why the Predictions Did Not Work

The matrix factorization performance is not great. The best NMF or SVD RMSE is 2-3 times the worst RMSE from the previous attempts, the approach where we predicted everything to 3. The RMSE for NMF gets slightly better as we increase the number of components, but then it starts to drop again.

The best NMF RMSE is 3.132. This means that on average each NMF prediction is 3.132 away from the corresponding observed value. On a 0-5 scale, that means we are more than half of the scale away on average for each prediction. This seems like a pretty poor value for RMSE. On a 0-5 scale, we would not want to trust these predictions.

SVD does not perform much better. For different numbers of components, SVD is sometimes higher and sometimes lower than NMF, but it is also worse than the broad attempt at predicting everything to 3.

The next question is why the NMF RMSE is so high.

What I think is going on is that we are working with very sparse data and that most of the utility matrix is 0s, 0s representing missing data. We are asking NMF to decompose that sparse matrix without having enough information to go on due to the sparsity. Also, NMF is treating 0s as rating values, as seen by 0s showing up as predictions. This results in NMF trying to pick up on patterns in the data that we do not have enough data to clearly define along with the zeros skewing the decomposition. We see some evidence that something is off with the predictions in the out-of-range values. NMF predicted ratings higher than 8 when the input ratings were on the 0-5 scale.

I am not sure how many component are typical before you need to up the `max_iter` parameter for NMF, but we had to do that around 15 latent features. 15 is lower than I was expecting. I think this is more evidence that NMF is struggling to decompose the utility matrix and needs more iterations available to estimate the W and H matrix approximations.

So, in the end, we can decompose the utility matrix into W and H, but the latent factors are not accurate enough due to the sparsity of the original utility matrix. When we multiply W and H together to rebuild the utility matrix with predictions in place of 0s, those predictions are all over the place, on average 3.132 away from what they should be on a 0-5 scale.

Here are some thoughts on addressing the sparsity issue.

We could reduce the dimensions of the utility matrix so that it is not quite as sparse. This might mean that we trim the users or the movies that we run at once. We may have ways to segment users into different groups and see if we end up with less sparsity within any of those groups. Similar for finding subsets of movies. We also could look at something like regularization on the original feature space to help identify in a more statistical manner rather than domain-knowledge manner which movies to leave out.

We could impute some values in place of 0s where appropriate, possibly by using domain knowledge or another form of prediction that is not as sweeping across the entire utility matrix. For example, we may be able to identify certain users or movies where we see stronger patterns, and we can use those to impute missing values with more certainty than estimating all missing values across the whole utility matrix at once.

Both of these approaches assume that we can use a subset of the data to refine patterns and then apply those patterns to the full data. The assumption in there is that the subsets are similar enough to the full dataset that the patterns are relevant to all the data. This may or may not be true. It would take some digging into when trying to find subsets or smaller scopes for imputation.

One last thought is to use algorithms that are better able to handle sparse matrices. One example would be to adjust the objective function for finding the W and H approximations. We might be able to adjust that to do something like penalize too many 0s or provide stronger weighting to non-0s. It does look like there are ways to incorporate constraints on the objective function. In a similar vein, we could look at algorithms that tackle the sparse matrix in a divide-and-conquer manner or similar that is able to ensemble together findings from subsets of the data that provide stronger signals for patterns in the data. This fits somewhat with what I mentioned a few paragraphs above, but there are also entire strategies that look like they are to address this issue of sparsity for matrix factorization.

In summary, using the base NMF from `sklearn` as we did here is not a good solution for this set of movie ratings. My sense is that it is because of the sparsity of the utility matrix. For now, we either need to find an alternative matrix factorization approach or use one of the similarity-based measures that we implemented in the week three assignment.

## References

I needed some help figuring this out. Here are some resources I used when getting up to speed on using matrix factorization for rating predictions.

Matrix factorization for predictions:
- https://medium.com/beek-tech/predicting-ratings-with-matrix-factorization-methods-cf6c68da775
- https://medium.com/analytics-vidhya/matrix-factorization-made-easy-recommender-systems-7e4f50504477
- https://medium.com/logicai/non-negative-matrix-factorization-for-recommendation-systems-985ca8d5c16c
- https://stackoverflow.com/questions/42357450/scikit-learn-non-negative-matrix-factorization-nmf-for-sparse-matrix

How to handle sparse data for NMF:
- https://jmlr.csail.mit.edu/papers/volume5/hoyer04a/hoyer04a.pdf
- https://faculty.cc.gatech.edu/~hpark/papers/GT-CSE-08-01.pdf